In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from typing import Tuple, List
import plotly.graph_objects as go
import tensorflow as tf
tf.keras.utils.set_random_seed(48)
tf.config.experimental.enable_op_determinism()

class TimeSeriesDataset(Dataset):
    def __init__(self, data: pd.DataFrame, sequence_length: int):
        self.data = torch.FloatTensor(data.values)
        self.sequence_length = sequence_length

    def __len__(self) -> int:
        return len(self.data) - self.sequence_length

    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, torch.Tensor]:
        return (
            self.data[idx:idx + self.sequence_length],
            self.data[idx + self.sequence_length]
        )

class GaussianLayer(nn.Module):
    def __init__(self, input_dim: int, output_dim: int):
        super().__init__()
        self.mu_layer = nn.Linear(input_dim, output_dim)
        self.sigma_layer = nn.Linear(input_dim, output_dim)
        
    def forward(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        mu = self.mu_layer(x)
        sigma = torch.log1p(torch.exp(self.sigma_layer(x))) + 1e-6
        return mu, sigma

class DeepAR(pl.LightningModule):
    def __init__(
        self,
        input_size: int = 1,
        hidden_size: int = 64,
        num_layers: int = 2,
        learning_rate: float = 1e-3
    ):
        super().__init__()
        self.save_hyperparameters()
        
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True
        )
        self.dense = nn.Linear(hidden_size, 32)
        self.activation = nn.ReLU()
        self.gaussian = GaussianLayer(32, input_size)
        self.learning_rate = learning_rate
        
    def gaussian_loss(
        self,
        y_true: torch.Tensor,
        mu: torch.Tensor,
        sigma: torch.Tensor
    ) -> torch.Tensor:
        pi_tensor = torch.tensor(2 * np.pi, device=self.device, dtype=torch.float32)
        return 0.5 * torch.mean(
            torch.log(pi_tensor)
            + torch.log(sigma)
            + (y_true - mu)**2 / (sigma**2)
        )

    def forward(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        lstm_out, _ = self.lstm(x)
        dense_out = self.activation(self.dense(lstm_out[:, -1, :]))
        mu, sigma = self.gaussian(dense_out)
        return mu, sigma

    def training_step(
        self,
        batch: Tuple[torch.Tensor, torch.Tensor],
        batch_idx: int
    ) -> torch.Tensor:
        x, y = batch
        mu, sigma = self(x)
        loss = self.gaussian_loss(y, mu, sigma)
        self.log('train_loss', loss, prog_bar=True)
        return loss

    def validation_step(
        self,
        batch: Tuple[torch.Tensor, torch.Tensor],
        batch_idx: int
    ) -> torch.Tensor:
        x, y = batch
        mu, sigma = self(x)
        loss = self.gaussian_loss(y, mu, sigma)
        self.log('val_loss', loss, prog_bar=True)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.learning_rate)


In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from typing import Tuple, List
import plotly.graph_objects as go
import time  # Import time module

# [Your existing classes: TimeSeriesDataset, GaussianLayer, DeepAR remain unchanged]

def prepare_data(filepath: str):
    try:
        df = pd.read_csv(filepath, index_col='Date/Time', parse_dates=True)
        df = df[df.index.minute == 0]
        df = pd.DataFrame({'120m': df['120m']}, index=df.index)
        
        mean = df.mean()
        std = df.std()
        df_normalized = (df - mean) / std
        
        # Split sizes
        train_size = 0.7
        val_size = 0.15
        test_size = 0.15
        
        # Calculate split indices
        n = len(df_normalized)
        train_end = int(n * train_size)
        val_end = train_end + int(n * val_size)
        
        # Split data
        train_data = df_normalized[:train_end]
        val_data = df_normalized[train_end:val_end]
        test_data = df_normalized[val_end:]
        
        return train_data, val_data, test_data, mean, std
        
    except Exception as e:
        print(f"Error loading data: {str(e)}")
        raise

def compute_picp_mpiw(y_true, y_lower, y_upper):
    """
    Compute Prediction Interval Coverage Probability (PICP) and Mean Prediction Interval Width (MPIW).
    
    Args:
        y_true (np.ndarray): True target values.
        y_lower (np.ndarray): Lower bounds of prediction intervals.
        y_upper (np.ndarray): Upper bounds of prediction intervals.
    
    Returns:
        picp (float): Prediction Interval Coverage Probability.
        mpiw (float): Mean Prediction Interval Width.
    """
    # Ensure inputs are flattened
    y_true = y_true.flatten()
    y_lower = y_lower.flatten()
    y_upper = y_upper.flatten()
    
    # Calculate coverage
    coverage = ((y_true >= y_lower) & (y_true <= y_upper)).mean()
    
    # Calculate average width
    width = (y_upper - y_lower).mean()
    
    return coverage, width

def main():
    try:
        # Parameters
        SEQUENCE_LENGTH = 24
        BATCH_SIZE = 32
        MAX_EPOCHS = 100
        
        # Prepare data
        train_data, val_data, test_data, mean, std = prepare_data(r"D:\drive-download-20240928T074710Z-001\Data\san francisco.csv")
        
        # Create datasets
        train_dataset = TimeSeriesDataset(train_data, SEQUENCE_LENGTH)
        val_dataset = TimeSeriesDataset(val_data, SEQUENCE_LENGTH)
        test_dataset = TimeSeriesDataset(test_data, SEQUENCE_LENGTH)
        
        # Create dataloaders (without multiprocessing)
        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
        test_loader = DataLoader(test_dataset, batch_size=1)  # Using batch_size=1 for sequential prediction
        
        # Initialize model
        model = DeepAR()
        
        # Configure trainer
        trainer = pl.Trainer(
            max_epochs=MAX_EPOCHS,
            accelerator='auto',
            devices=1,
            callbacks=[
                pl.callbacks.EarlyStopping(
                    monitor='val_loss',
                    patience=10,
                    mode='min'
                ),
                pl.callbacks.ModelCheckpoint(
                    monitor='val_loss',
                    mode='min',
                    filename='{epoch}-{val_loss:.2f}'
                )
            ]
        )
        
        # Measure start time
        start_time = time.time()
        
        # Train model
        trainer.fit(model, train_loader, val_loader)
        
        # Measure end time
        end_time = time.time()
        
        # Calculate training time
        training_time_seconds = end_time - start_time
        training_time_minutes = training_time_seconds / 60
        
        print(f"Total Training Time: {training_time_seconds:.2f} seconds ({training_time_minutes:.2f} minutes)")
        
        # Switch to evaluation mode
        model.eval()
        
        # Lists to store all predictions and true values
        all_y_true = []
        all_y_pred = []
        all_y_lower = []
        all_y_upper = []
        
        # Iterate over the test_loader to collect predictions
        for batch in test_loader:
            x, y = batch  # x: [1, SEQUENCE_LENGTH, 1], y: [1, 1]
            with torch.no_grad():
                mu, sigma = model(x)  # mu and sigma: [1, 1]
            
            # Denormalize the data
            y_true_denorm = y.numpy() * std.values + mean.values
            y_pred_denorm = mu.cpu().numpy() * std.values + mean.values
            y_lower_denorm = (mu - 2 * sigma).cpu().numpy() * std.values + mean.values
            y_upper_denorm = (mu + 2 * sigma).cpu().numpy() * std.values + mean.values
            
            # Append to lists
            all_y_true.append(y_true_denorm)
            all_y_pred.append(y_pred_denorm)
            all_y_lower.append(y_lower_denorm)
            all_y_upper.append(y_upper_denorm)
        
        # Convert lists to numpy arrays
        all_y_true = np.array(all_y_true)
        all_y_pred = np.array(all_y_pred)
        all_y_lower = np.array(all_y_lower)
        all_y_upper = np.array(all_y_upper)
        
        # Calculate PICP and MPIW
        picp, mpiw = compute_picp_mpiw(all_y_true, all_y_lower, all_y_upper)
        print(f"PICP: {picp*100:.2f}%")
        print(f"MPIW: {mpiw:.4f}")
        
        # For plotting, you might want to plot a subset (e.g., first 100 predictions)
        plot_length = min(100, len(all_y_true))
        y_true_plot = all_y_true[:plot_length]
        y_pred_plot = all_y_pred[:plot_length]
        y_lower_plot = all_y_lower[:plot_length]
        y_upper_plot = all_y_upper[:plot_length]
        
        # Plot results with Prediction Intervals
        fig = go.Figure()
        x = np.arange(plot_length)
        
        fig.add_trace(go.Scatter(x=x, y=y_true_plot.flatten(), name='True Values', line=dict(color='blue')))
        fig.add_trace(go.Scatter(x=x, y=y_pred_plot.flatten(), name='Predictions', line=dict(color='green')))
        fig.add_trace(go.Scatter(
            x=np.concatenate([x, x[::-1]]),
            y=np.concatenate([y_upper_plot.flatten(), y_lower_plot.flatten()[::-1]]),
            fill='toself',
            fillcolor='rgba(128,128,128,0.2)',
            line=dict(color='rgba(255,255,255,0)'),
            name='95% Prediction Interval'
        ))
        
        fig.update_layout(
            title=f'Time Series Predictions (PICP: {picp*100:.2f}%, MPIW: {mpiw:.4f})',
            xaxis_title='Time Step',
            yaxis_title='Value'
        )
        fig.show()
        
    except Exception as e:
        print(f"Error in main execution: {str(e)}")
        raise

if __name__ == "__main__":
    main()


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name       | Type          | Params | Mode 
-----------------------------------------------------
0 | lstm       | LSTM          | 50.4 K | train
1 | dense      | Linear        | 2.1 K  | train
2 | activation | ReLU          | 0      | train
3 | gaussian   | GaussianLayer | 66     | train
-----------------------------------------------------
52.6 K    Trainable params
0         Non-trainable params
52.6 K    Total params
0.210     Total estimated model params size (MB)
6         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

d:\Python3.11\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


d:\Python3.11\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Epoch 17: 100%|██████████| 575/575 [00:15<00:00, 37.23it/s, v_num=4, train_loss=0.647, val_loss=0.865]
Total Training Time: 258.09 seconds (4.30 minutes)
PICP: 98.67%
MPIW: 6.3316
